In [2]:
head -n 30 steps/align_sgmm2.sh

#!/bin/bash
# Copyright 2012  Johns Hopkins University (Author: Daniel Povey)
# Apache 2.0

# Computes training alignments and (if needed) speaker-vectors, given an
# SGMM system.  If the system is built on top of SAT, you should supply
# transforms with the --transform-dir option.

# If you supply the --use-graphs option, it will use the training
# graphs from the source directory.

# Begin configuration section.
stage=0
nj=8
cmd=run.pl
use_graphs=false # use graphs from srcdir
use_gselect=false # use gselect info from srcdir [regardless, we use
   # Gaussian-selection info, we might have to compute it though.]
gselect=15  # Number of Gaussian-selection indices for SGMMs.
# Begin configuration.
scale_opts="--transition-scale=1.0 --acoustic-scale=0.1 --self-loop-scale=0.1"
beam=10
retry_beam=40
transform_dir=  # directory to find fMLLR transforms in.
# End configuration options.

echo "$0 $@"  # Print the command line for logging

[ -f ${KALDI_PATH}/path.sh ] && . ${KALDI_PATH}/path.sh

In [3]:
mkdir -p exp/asr_aligned

In [11]:
compile-train-graphs

bash: compile-train-graphs: command not found


: 127

In [12]:
mkdir -p exp/asr_mono_ali
steps/align_si.sh --nj 8 data/asr_dir data/lang exp/tri4 exp/asr_mono_ali

steps/align_si.sh --nj 8 data/asr_dir data/lang exp/tri4 exp/asr_mono_ali
steps/align_si.sh: expected file data/asr_dir/text to exist


: 1

In [10]:
steps/align_sgmm2.sh --nj 8 --transform-dir exp/tri3 data/asr_dir data/lang exp/sgmm2_4 exp/asr_aligned

steps/align_sgmm2.sh --nj 8 --transform-dir exp/tri3 data/asr_dir data/lang exp/sgmm2_4 exp/asr_aligned
steps/align_sgmm2.sh: feature type is lda
steps/align_sgmm2.sh: using transforms from exp/tri3
steps/align_sgmm2.sh: compiling training graphs
run.pl: 8 / 8 failed, log is in exp/asr_aligned/log/compile_graphs.*.log


: 1